<a href="https://colab.research.google.com/github/FaiazS/Handwritten-Digit-Recognition-using-CNN/blob/main/Handwritten_Digit_Recognition_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import wandb

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical

from wandb.integration.keras import WandbMetricsLogger

In [15]:
#Generating a unique group name for each run

project_name = f"mnist-execution-{int(time.time())}"  #Using timestamp as an unique identifier

In [16]:
#Initalizing W&B

wandb.init(project = project_name,config = {"epochs" : 10, "batch_size" : 32})

In [18]:
#Loading and Preprocessing MNIST Dataset

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

#Reshaping the data to fit the CNN input(Adding Channel Dimensions)

x_train = x_train.reshape((x_train.shape[0],28,28,1)).astype('float32')/ 255.0

x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32') / 255.0

#One hot encoding the labels

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [21]:
#Build the CNN Model

cnn_model = models.Sequential([

                               #Input Layer
                               tf.keras.Input(shape =(28, 28, 1)),

                               #First Convolutional Layer
                               tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),

                               tf.keras.layers.MaxPooling2D((2, 2)),

                               #Second Convolutional Layer
                               tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),

                               tf.keras.layers.MaxPooling2D((2, 2)),

                               tf.keras.layers.Flatten(), #Because Dense layer expects input in 1D format.

                               #Fully Connected Layer

                               tf.keras.layers.Dense(128, activation ='relu'),

                               tf.keras.layers.Dropout(0.5),                  # To prevent overfitting and Improve the Generalization ability of the model.

                               tf.keras.layers.Dense(10, activation = 'softmax')  #Output layer for 10 classes


                               ])

In [22]:
cnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
#Compling the CNN Model

cnn_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [30]:
#Training the CNN Model

cnn_model_training_stats = cnn_model.fit(x_train, y_train, validation_data = (x_test, y_test), validation_split = 0.2, callbacks = [WandbMetricsLogger()], epochs = 7, batch_size = 32)

Epoch 1/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 29ms/step - accuracy: 0.8545 - loss: 0.4466 - val_accuracy: 0.9851 - val_loss: 0.0485
Epoch 2/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 29ms/step - accuracy: 0.9735 - loss: 0.0899 - val_accuracy: 0.9881 - val_loss: 0.0384
Epoch 3/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 29ms/step - accuracy: 0.9809 - loss: 0.0611 - val_accuracy: 0.9907 - val_loss: 0.0294
Epoch 4/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.9849 - loss: 0.0482 - val_accuracy: 0.9898 - val_loss: 0.0312
Epoch 5/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.9878 - loss: 0.0386 - val_accuracy: 0.9912 - val_loss: 0.0255
Epoch 6/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 52s 28ms/step - accuracy: 0.9892 - loss: 0.0339 - val_accuracy: 0.9914 - val_loss: 0.0278
Epoch 7/7
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 27ms/step - accuracy: 0.9909 - loss: 0.0282 - val_accuracy: 0.9926 - val_loss: 0.0238


In [38]:
#Evaluating the CNN Model

test_loss, test_accuracy = cnn_model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9907 - loss: 0.0290


In [39]:
#Outputting the Validation/Test Accuracy

print(f"Test accuracy:",{test_accuracy})

Test accuracy: {0.9926000237464905}
